In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import logging
from scipy.interpolate import RectBivariateSpline
from functions import load_test_data
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
except RuntimeError as e:
    logging.info(e)

In [2]:
#3flavor_clean, 3flavor_poisson, nsi_clean, nsi_poisson
learn_target = '3flavor_poisson'

dnn = tf.keras.models.load_model('./dnn/' + learn_target + '/1.h5')
data_all = load_test_data(learn_target)

In [3]:
if learn_target == '3flavor_clean':
    IO_prediction = dnn.predict(np.array([data_all[0]]))[0]
    NO_prediction = dnn.predict(np.array([data_all[1]]))[0]

    IO_delta = np.angle(IO_prediction[2] +1j*IO_prediction[3])
    NO_delta = np.angle(NO_prediction[2] +1j*NO_prediction[3])
    if IO_delta < 0: IO_delta += 2*np.pi
    if NO_delta < 0: NO_delta += 2*np.pi
    IO_prediction = np.array([IO_prediction[0], IO_prediction[1], IO_delta])*180/np.pi
    NO_prediction = np.array([NO_prediction[0], NO_prediction[1], NO_delta])*180/np.pi

    print('IO prediction = ', IO_prediction)
    print('NO prediction = ', NO_prediction)

In [7]:
if learn_target == '3flavor_poisson':
    sample_amount = 100000
    data_IO = np.random.poisson(data_all[1], (sample_amount, 144))
    data_NO = np.random.poisson(data_all[0], (sample_amount, 144))

    prediction_IO = dnn.predict(data_IO)
    prediction_NO = dnn.predict(data_NO)

    np.savez('./IO_contour/' + learn_target + '/dnn.npz', prediction_IO)
    np.savez('./NO_contour/' + learn_target + '/dnn.npz', prediction_NO)